<a href="https://www.kaggle.com/code/esantosdata/titanic-dataset-applying-ml-model-on-datas?scriptVersionId=222897392" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as num
train_data = pd.read_csv("/kaggle/input/titanic/train.csv") #Ler o arquivo
train_data.head() #Mostrar  cabeçalho as primeiras 5  linhas usando  head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


**EXPLORANDO PADROES**

Lembre-se que o arquivo "gender_submission.csv" pressupoe que todas as passageiras femininas sobreviveram e todos os passageiros masculinos morreram. Vamos agora verificar se esse padrao e verdadeiro no arquivo "train_csv".

In [3]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% de mulheres que sobreviveram:", rate_women)

% de mulheres que sobreviveram: 0.7420382165605095


In [4]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% de homens que sobreviveram:", rate_men)

% de homens que sobreviveram: 0.18890814558058924


**A partir daqui podemos verificar que cerca de 75% das mulheres sobreviveram e 18% dos homens sobreviveram. O genero pode ser um forte indicador de sobrevivencia,logo o arquivo "gender_submission.csv" nao e um mau palpite para comecarmos.**

In [5]:
gender_submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
gender_submission.head()  # Apenas para visualização

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


**Este modelo como vimos acima baseia suas previsoes em apenas uma coluna, como você pode imaginar, ao considerar várias colunas, podemos descobrir padrões mais complexos que podem potencialmente produzir previsões mais bem informadas. Como é muito difícil considerar várias colunas ao mesmo tempo (ou levaria muito tempo para considerar todos os padrões possíveis em muitas colunas diferentes simultaneamente), usaremos M.L para automatizar isso para nós.**

* **Vamos construir o modelo que e conhecido como Random Forest Model, este modelo é construído de várias "árvores" (há três árvores na imagem abaixo, mas vamos construir 100!) que considerarão individualmente os dados de cada passageiro e votarão se o indivíduo sobreviveu. Então, o modelo de floresta aleatória toma uma decisão democrática: o resultado com mais votos vence!**
* * https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
![](https://infoaryan.com/wp-content/uploads/2023/11/random-forest-diagram-768x525.jpg)


In [6]:
from sklearn.ensemble import RandomForestClassifier #
y = train_data["Survived"] 

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


***## ABAIXO TEMOS A EXPLICAÇAO DE CADA LINHA DA IMPLEMENTAÇAO DO MODELO DE M.L APLICADO NA NOSSA ANALISE.*****

**1. Importação do modelo de Random Forest:**
   Aqui, estamos importando o RandomForestClassifier da biblioteca sklearn (Scikit-learn), que é uma das bibliotecas mais usadas para machine   learning em Python. Esse modelo vai ser usado para construir uma floresta de árvores de decisão para prever se um passageiro sobreviveu ou não.

**2. Definição da variável target (y) :**
   train_data["Survived"] acessa a coluna Survived do conjunto de dados train_data, que é a variável de saída que queremos prever.
y será o vetor de rótulos (a resposta), ou seja, se o passageiro sobreviveu (1) ou não (0).

**3. Definição das variáveis independentes (features):**
   Aqui você está selecionando as colunas que serão usadas como características de entrada (variáveis independentes) para treinar o modelo.

    "Pclass": Classe do passageiro (1, 2 ou 3)
    "Sex": Gênero do passageiro (masculino ou feminino)
    "SibSp": Número de irmãos/cônjuges a bordo
    "Parch": Número de pais/filhos a bordo

**4. Transformação das variáveis categóricas :**
   O método pd.get_dummies() converte as variáveis categóricas (como "Sex") em variáveis numéricas, criando uma coluna para cada valor único. Por exemplo, "Sex" será transformado em duas colunas, uma para "female" e outra para "male", com 0 ou 1 indicando a presença do valor.
X é o conjunto de dados de entrada (features) após a conversão de variáveis categóricas em variáveis numéricas.

**5. Transformação das variáveis de teste :**
   Aqui, a mesma transformação de variáveis categóricas é aplicada ao conjunto de dados de teste test_data, de forma a garantir que as colunas de entrada (X_test) tenham o mesmo formato de X (conjunto de treinamento).

**6. Criação e treinamento do modelo Random Forest :**
   n_estimators=100: Define o número de árvores na floresta. O modelo irá usar 100 árvores de decisão.
max_depth=5: Define a profundidade máxima das árvores. Limita a quantidade de divisões em cada árvore, ajudando a evitar overfitting.
random_state=1: Define uma semente para a aleatoriedade, garantindo que o resultado seja reprodutível (sempre que rodar o código com a mesma semente, o modelo terá o mesmo comportamento).

**7. Treinamento do modelo :**
   model.fit(X, y) treina o modelo usando as variáveis de entrada (X) e a variável de saída (y), ou seja, ele aprende a relação entre as features e o resultado de sobrevivência com base no conjunto de dados de treinamento.

**8. Fazendo previsões com o modelo treinado :**
   model.predict(X_test) usa o modelo treinado para fazer previsões no conjunto de dados de teste X_test. O modelo tentará prever se os passageiros do conjunto de dados de teste sobreviveram ou não.

**9. Criando o DataFrame de saída :**
   Aqui você cria um DataFrame chamado output que contém duas colunas:

    PassengerId: Identificador de cada passageiro.
    Survived: As previsões feitas pelo modelo (0 ou 1, dependendo se o modelo previu que o passageiro sobreviveu ou não).

**10. Salvando a saída em um arquivo CSV :**
    output.to_csv('submission.csv', index=False) salva o DataFrame output em um arquivo CSV chamado "submission.csv". O parâmetro index=False significa que o índice das linhas não será incluído no arquivo.

**11. Mensagem de confirmação :**
    Por fim, o código imprime uma mensagem de sucesso, indicando que o arquivo foi salvo com sucesso.

# *Medindo a acuracia deste modelo resultamos em : 0.77511 - Oque nos diz que fomos cerca de 70% acertivos na taxa de sobrevivencia, podemos ainda irmos mais profundamente nestes dados entao seguimos explorando este dataset utilizando outras ferramentas e novas formas de exploraçao em busca de uma acertividade melhor, implementando alguns conceitos de estatistica, visualizaçao de dados, limpeza de dados e aplicaçao de modelos de machine learning.*

In [7]:
# Para uma melhor visualizaçao da base de dados, vamos utilizar o ydata-profiling (antigo pandas profiling), Podemos tambem utilizar o Excel,SQL,PQ...
from ydata_profiling import ProfileReport

In [8]:
# Gerando visualizaçao
profile = ProfileReport(train_data, title="Titanic_train")

In [9]:
# Criando relatorio, exportando o HTML para a pasta.
# Neste momento pode apresentar uma mensagem de erro mas nao impede a geracao do relatorio, isto acontece pois deveriamos utilizar apos o tratamento de dados "NaN", Porem a biblioteca foi carregada em add-ons do notebook, e foi a ferramenta escolhida parar uma primeira analise grafica dos dados.
profile.to_file("Titanic_train.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# Analise da base :
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
# Verificando os tipos de dados : 
train_data.dtypes.value_counts()

int64      5
object     5
float64    2
Name: count, dtype: int64

In [12]:
# Verificando valores nulos :
train_data.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

# Agora vamos verificar a base de testes :

In [13]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [14]:
test_data.dtypes.value_counts()

object     5
int64      4
float64    2
Name: count, dtype: int64

In [15]:
test_data.isnull().sum().sort_values(ascending=False).head(5)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
dtype: int64

* Temos colunas que **possuem valores vazios na base de testes e que nao estao vazias na base de treino** (neste caso, precisariamos tratar estas colunas apenas na base de teste.)

## REALIZANDO OS TRATAMENTOS INICIAIS DOS DADOS  : 

* Podemos comecar tratando os valores vazios

In [16]:
# Verificando novamente os valores vazios da base de treino :
train_data.isnull().sum().sort_values(ascending=False).head(5)


Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

In [17]:
# Retornando o shape da base :
train_data.shape

(891, 12)

In [18]:
# Visualizando novamente a base : 
train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
# Verificando a cardinalidade dos dados : 
train_data.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Pclass           3
Embarked         3
Survived         2
Sex              2
dtype: int64

* Colunas com **alta cardinalidade** como nome, numero do ticket e codigo da cabine nao vao ajudar o modelo pois podem torna-lo **menos generalizavel.** Nesse caso vamos **eliminar estas colunas**

## Nota importante : Por exemplo, se assumirmos que o passageiro com o ID = 5, Henry Willian Allen vai sobreviver, nos nao conseguimos fazer uma previsao para novos dados, pois muito provavelmente nao tera um passageiro com o mesmo nome. Mais para a frente, podemos sim, utilizar estas colunas para nos ajudar a sermos mais assertivos. 

In [20]:
# Eliminando na base de treino
train_data = train_data.drop(['Name','Ticket','Cabin'],axis=1)

In [21]:
# Eliminando na base de teste
test_data = test_data.drop(['Name','Ticket','Cabin'],axis=1)

* Realizamos este drop em ambas as bases, porque nao podemos ter bases diferentes para enviar ao nosso modelo, precisamos treinar com a mesma estrutura que vamos avaliar o nosso modelo de teste. 

* Agora precisamos tratar a coluna "Age", que possui valores vazios tanto no train_data quanto no test_data
    * Como esta coluna possui a idade dos passageiros, podemos utilizar a **media das idades** para substituir os valores vazios.

In [22]:
# Descobrindo a media de idade para a base train_data : 
train_data.Age.mean()

29.69911764705882

In [23]:
# Podemos agora selecionar as colunas que estao com valores vazios e substituir pela media de idade que encontramos.

#train_data["Age"].fillna(train_data["Age"].mean(), inplace=True)
#train_data["Age"] = train_data["Age"].fillna(train_data["Age"].mean()) 
train_data.loc[train_data.Age.isnull(),'Age'] = train_data.Age.mean()


In [24]:
# O mesmo para a base test_data : 
test_data.loc[test_data.Age.isnull(),'Age'] = test_data.Age.mean()

* Para a base de treino, ainda temos 2 valores vazios na coluna **Embarked**.
    * Podemos utilizar a mesma logica que usamos para "Age" e substituir pela **moda**.

In [25]:
# Verificando os valores da coluna Embarked :
train_data.Embarked.value_counts()


Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [26]:
# Selecionando a moda da coluna : 
train_data.Embarked.mode()[0]

'S'

In [27]:
# Substituindo os valores vazios pelo o mais frequente : 
train_data.loc[train_data.Embarked.isnull(),'Embarked'] = train_data.Embarked.mode()[0]


* Para finalizar a base de teste temos ainda 1 valor vazio em **"Fare"**
     * Vamos utilizar a mesma logica da coluna "Age" e substituir pela **moda**

In [28]:
# Verificando e substituindo a media da coluna Fare para a base de teste : 
test_data.loc[test_data.Fare.isnull(),'Fare'] = test_data.Fare.mean()

## Apos todos estes tratamentos vamos verificar novamente os valores vazios de nossa base

In [29]:
# Para a base de treino
train_data.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
dtype: int64

In [30]:
# Para a base de teste
test_data.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
dtype: int64

## Considerando apenas as colunas que nao sao de texto. 

In [31]:
# Verificando as colunas de texto na base train_data
train_data.columns[train_data.dtypes != 'object']

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [32]:
# Selecionando somente os valores numericos da base train_data 
train_data_nr = train_data.loc[:,train_data.columns[train_data.dtypes != 'object']]

In [33]:
# Verificando as colunas de texto na base test_data
test_data.columns[test_data.dtypes != 'object']

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [34]:
# Selecionando somente os valores numericos da base test_data
test_data_nr = test_data.loc[:,test_data.columns[test_data.dtypes != 'object']]

E valido comentar que na base train_data temos a coluna "Survived" e na base test_data nao. Pois e justamente esta coluna que iremos prever.

## Podemos agora selecionar um modelo para classificar esses dados.

Vamos testar estes 3 modelos primeiramente :
1. Arvore de classificacao
   https://scikit-learn.org/stable/modules/tree.html#classification
3. Classificacao dos vizinhos mais proximos https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
4. Regressao Logistica https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

* Antes de usarmos estes algoritmos, precisamos separar a base_train em train_val
    * Vamos fazer isso utilizando o train_test_split https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [35]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [36]:
# Separando a base_train em x e y
X = train_data_nr.drop(['PassengerId','Survived'],axis=1)
y = train_data.Survived

In [37]:
# Separando em treino e validaçao
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

* Para a **Arvore de Classificaçao**

In [38]:
# Fazendo a importaçao
from sklearn import tree

In [39]:
# Criando classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [40]:
# Fazendo fit com os dados
clf_ac = clf_ac.fit(X_train, y_train)

In [41]:
# Fazendo a previsao
y_pred_ac = clf_ac.predict(X_val)

* Para o **KNeighborsClassifier**

In [42]:
# Fazendo a importaçao
from sklearn.neighbors import KNeighborsClassifier

In [43]:
# Criando classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [44]:
# Fazendo fit com os dados
clf_knn = clf_knn.fit(X_train, y_train)

In [45]:
# Fazendo a previsao
y_pred_knn = clf_knn.predict(X_val)

* E para a **Regressao logistica**

In [46]:
# Fazendo a importaçao
from sklearn.linear_model import LogisticRegression

In [47]:
# Criando classificador
clf_rl = LogisticRegression(random_state=42)

In [48]:
# Fazendo fit com os dados
clf_rl = clf_rl.fit(X_train, y_train)

In [49]:
# Fazendo a previsao
y_pred_rl = clf_rl.predict(X_val)

## E PARA FINALIZAR, VAMOS AVALIAR ESTES MODELOS

* Para fazermos esta analise, podemos usar:
  * **Acuracia** (metodo de avaliaçao da competiçao): https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
  * **Matriz de confusao** (ajuda a visualizar a distribuiçao dos erros): https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

* **Acuracia:**

In [50]:
# Importando
from sklearn.metrics import accuracy_score

In [51]:
# Para a Arvore de classificaçao
accuracy_score(y_val, y_pred_ac)

0.6169491525423729

In [52]:
# Para o KNN
accuracy_score(y_val, y_pred_knn)

0.6610169491525424

In [53]:
# Para a Regressao logistica
accuracy_score(y_val, y_pred_rl)

0.7254237288135593

* **Matriz de confusao:**

In [54]:
# Importando
from sklearn.metrics import confusion_matrix

In [55]:
# Para a Arvore de classificaçao
confusion_matrix(y_val, y_pred_ac)

array([[125,  50],
       [ 63,  57]])

In [56]:
# Para o KNN
confusion_matrix(y_val, y_pred_knn)

array([[133,  42],
       [ 58,  62]])

In [57]:
# Para a Regressao logistica
confusion_matrix(y_val, y_pred_rl)

array([[156,  19],
       [ 62,  58]])

## Fazendo a previsao para dos dados de teste
       * Vamos usar o modelo com melhor precisao para fazer o predict na base de teste 

In [58]:
# Visualizando o X_train
X_train.head(5)

,Pclass,Age,SibSp,Parch,Fare
6,1,54.000000,0,0,51.8625
718,3,29.699118,0,0,15.5000
685,2,25.000000,1,2,41.5792
73,3,26.000000,1,0,14.4542
882,3,22.000000,0,0,10.5167


In [59]:
# Visualizando a base de teste (apenas com valores numericos)
train_data_nr.head(5)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500


In [60]:
# Para a base de teste ser igual a base de treino, retiramos a coluna 'PassengerId'
X_test_data = test_data_nr.drop("PassengerId",axis=1)

In [61]:
# Utilizando a regressao logistica na base teste
y_pred = clf_rl.predict(X_test_data)

In [62]:
# Criando uma nova coluna na previsao na base de teste
test_data['Survived'] = y_pred

In [63]:
# Selecionando apenas a colunia PassengerId e Survived para fazer o envio
base_envio = test_data[['PassengerId','Survived']]

In [64]:
# Exportando para um CSV
base_envio.to_csv('resultados2.csv',index=False)

# Verificando a acuracia temos 0.66746 cerca de 66% de acuracia neste modelo de Regressao logistica